# Financial Analytics Dashboard

This notebook provides an analytical view of all financial statements with interactive dropdowns for bank and account selection.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Database connection function
def connect_to_db():
    """Connect to the SQLite database"""
    try:
        conn = sqlite3.connect('../data/finance.db')
        return conn
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None

# Test connection
conn = connect_to_db()
if conn:
    print("Database connection successful!")
    conn.close()
else:
    print("Database connection failed!")

In [ ]:
# Function to get available banks and accounts
def get_banks_and_accounts():
    """Get all available banks and investment accounts"""
    conn = connect_to_db()
    if not conn:
        return pd.DataFrame(), pd.DataFrame()
    
    # Get investment accounts - using correct column names
    investment_accounts = pd.read_sql_query("""
        SELECT id, name, custodian, account_type, created_at
        FROM investment_accounts
        ORDER BY custodian, name
    """, conn)
    
    # Get regular accounts (if they exist)
    try:
        regular_accounts = pd.read_sql_query("""
            SELECT id, name, custodian, account_type, created_at
            FROM accounts
            ORDER BY custodian, name
        """, conn)
    except:
        regular_accounts = pd.DataFrame()
    
    conn.close()
    return investment_accounts, regular_accounts

# Display available accounts
investment_accounts, regular_accounts = get_banks_and_accounts()

print("=== Investment Accounts ===")
if not investment_accounts.empty:
    display(investment_accounts)
else:
    print("No investment accounts found")

print("\n=== Regular Bank Accounts ===")
if not regular_accounts.empty:
    display(regular_accounts)
else:
    print("No regular bank accounts found")

In [ ]:
# Function to get portfolio statements for a specific account
def get_portfolio_statements(account_id):
    """Get all portfolio statements for a specific investment account"""
    conn = connect_to_db()
    if not conn:
        return pd.DataFrame()
    
    statements = pd.read_sql_query("""
        SELECT statement_date, opening_balance, period_gain_loss, ending_balance,
               total_market_value, total_cost_basis, total_unrealized_gain_loss
        FROM portfolio_statements
        WHERE investment_account_id = ?
        ORDER BY statement_date DESC
    """, conn, params=[account_id])
    
    conn.close()
    return statements

# Function to get securities for a specific statement
def get_securities(account_id, statement_date):
    """Get securities for a specific account and statement date"""
    conn = connect_to_db()
    if not conn:
        return pd.DataFrame()
    
    securities = pd.read_sql_query("""
        SELECT symbol, name, security_type, quantity, share_price,
               total_cost, market_value, unrealized_gain_loss
        FROM securities
        WHERE investment_account_id = ? AND statement_date = ?
        ORDER BY market_value DESC
    """, conn, params=[account_id, statement_date])
    
    conn.close()
    return securities

In [ ]:
# Interactive account selection
from ipywidgets import interact, Dropdown, HTML, VBox, HBox, Output
import ipywidgets as widgets

# Create dropdowns
account_dropdown = Dropdown(
    options=[],
    description='Select Account:',
    style={'description_width': 'initial'}
)

statement_dropdown = Dropdown(
    options=[],
    description='Select Statement:',
    style={'description_width': 'initial'}
)

# Output areas
summary_output = Output()
securities_output = Output()
charts_output = Output()

# Update account dropdown - MODIFIED to show ALL accounts
def update_account_dropdown():
    investment_accounts, _ = get_banks_and_accounts()
    if not investment_accounts.empty:
        # Show all accounts, even those without data
        options = [(f"{row['custodian'] or 'Unknown'} - {row['name']} (ID: {row['id']})", row['id']) 
                   for _, row in investment_accounts.iterrows()]
        account_dropdown.options = options
        if options:
            account_dropdown.value = options[0][1]
            print(f"✅ Loaded {len(options)} accounts in dropdown")
            print(f"📊 Accounts with data: {len([opt for opt in options if get_portfolio_statements(opt[1]).shape[0] > 0])}")
    else:
        account_dropdown.options = []

# Update statement dropdown
def update_statement_dropdown(account_id):
    if account_id:
        statements = get_portfolio_statements(account_id)
        if not statements.empty:
            options = [(row['statement_date'], row['statement_date']) 
                       for _, row in statements.iterrows()]
            statement_dropdown.options = options
            if options:
                statement_dropdown.value = options[0][1]
                print(f"✅ Found {len(options)} statements for account {account_id}")
        else:
            statement_dropdown.options = []
            print(f"⚠️  No statements found for account {account_id}")
    else:
        statement_dropdown.options = []

# Initialize dropdowns
update_account_dropdown()

In [ ]:
# Function to display portfolio summary
def display_portfolio_summary(account_id, statement_date):
    """Display portfolio summary for selected account and statement date"""
    with summary_output:
        summary_output.clear_output()
        
        if not account_id or not statement_date:
            return
        
        statements = get_portfolio_statements(account_id)
        if statements.empty:
            display(HTML("<p style='color: red;'>⚠️ No portfolio statements found for this account.</p>"))
            return
            
        statement = statements[statements['statement_date'] == statement_date].iloc[0]
        
        # Get account details
        investment_accounts, _ = get_banks_and_accounts()
        account = investment_accounts[investment_accounts['id'] == account_id].iloc[0]
        
        html_content = f"""
        <div style='background-color: #f8f9fa; padding: 20px; border-radius: 10px; border-left: 5px solid #007bff;'>
            <h3 style='color: #007bff; margin-top: 0;'>{account['custodian'] or 'Unknown'} - {account['name']}</h3>
            <h4 style='color: #495057;'>Statement Period: {statement_date}</h4>
            <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px;'>
                <div>
                    <p><strong>Opening Balance:</strong> ${statement['opening_balance']:,.2f}</p>
                    <p><strong>Period Gain/Loss:</strong> ${statement['period_gain_loss']:,.2f}</p>
                    <p><strong>Ending Balance:</strong> ${statement['ending_balance']:,.2f}</p>
                </div>
                <div>
                    <p><strong>Total Market Value:</strong> ${statement['total_market_value']:,.2f}</p>
                    <p><strong>Total Cost Basis:</strong> ${statement['total_cost_basis']:,.2f}</p>
                    <p><strong>Total Unrealized G/L:</strong> ${statement['total_unrealized_gain_loss']:,.2f}</p>
                </div>
            </div>
        </div>
        """
        
        display(HTML(html_content))

In [ ]:
# Function to display securities
def display_securities(account_id, statement_date):
    """Display securities for selected account and statement date"""
    with securities_output:
        securities_output.clear_output()
        
        if not account_id or not statement_date:
            return
        
        securities = get_securities(account_id, statement_date)
        
        if not securities.empty:
            # Calculate additional metrics
            securities['weight'] = (securities['market_value'] / securities['market_value'].sum() * 100).round(2)
            securities['return_pct'] = (securities['unrealized_gain_loss'] / securities['total_cost'] * 100).round(2)
            
            # Display securities table
            display(HTML(f"<h4>Securities ({len(securities)} holdings)</h4>"))
            display(securities[['symbol', 'name', 'quantity', 'share_price', 'market_value', 'weight', 'return_pct']])
        else:
            display(HTML("<p style='color: red;'>⚠️ No securities found for this statement period.</p>"))

In [ ]:
# Function to create charts
def create_charts(account_id, statement_date):
    """Create charts for portfolio analysis"""
    with charts_output:
        charts_output.clear_output()
        
        if not account_id or not statement_date:
            return
        
        securities = get_securities(account_id, statement_date)
        
        if not securities.empty:
            # Create figure with subplots
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
            fig.suptitle(f'Portfolio Analysis - {statement_date}', fontsize=16, fontweight='bold')
            
            # 1. Portfolio Allocation by Market Value
            top_securities = securities.nlargest(10, 'market_value')
            ax1.pie(top_securities['market_value'], labels=top_securities['symbol'], 
                   autopct='%1.1f%%', startangle=90)
            ax1.set_title('Portfolio Allocation (Top 10)')
            
            # 2. Return Performance
            colors = ['green' if x >= 0 else 'red' for x in securities['return_pct']]
            ax2.barh(securities['symbol'], securities['return_pct'], color=colors)
            ax2.set_xlabel('Return (%)')
            ax2.set_title('Individual Security Returns')
            ax2.axvline(x=0, color='black', linestyle='-', alpha=0.3)
            
            # 3. Market Value vs Cost Basis
            ax3.scatter(securities['total_cost'], securities['market_value'], 
                       s=securities['market_value']/100, alpha=0.7)
            ax3.plot([securities['total_cost'].min(), securities['total_cost'].max()], 
                    [securities['total_cost'].min(), securities['total_cost'].max()], 
                    'r--', alpha=0.5, label='Cost = Market Value')
            ax3.set_xlabel('Cost Basis')
            ax3.set_ylabel('Market Value')
            ax3.set_title('Market Value vs Cost Basis')
            ax3.legend()
            
            # 4. Security Type Distribution
            if 'security_type' in securities.columns:
                type_counts = securities['security_type'].value_counts()
                ax4.pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
                ax4.set_title('Security Type Distribution')
            
            plt.tight_layout()
            plt.show()
        else:
            display(HTML("<p style='color: red;'>⚠️ No securities data available for charts.</p>"))

In [ ]:
# Event handlers for dropdowns
def on_account_change(change):
    if change['new']:
        print(f"🔄 Account changed to ID: {change['new']}")
        update_statement_dropdown(change['new'])
        if statement_dropdown.value:
            display_portfolio_summary(change['new'], statement_dropdown.value)
            display_securities(change['new'], statement_dropdown.value)
            create_charts(change['new'], statement_dropdown.value)
        else:
            # Clear outputs if no statements
            with summary_output:
                summary_output.clear_output()
                display(HTML("<p style='color: orange;'>📝 This account has no portfolio statements yet.</p>"))
            with securities_output:
                securities_output.clear_output()
            with charts_output:
                charts_output.clear_output()

def on_statement_change(change):
    if change['new'] and account_dropdown.value:
        print(f"🔄 Statement changed to: {change['new']}")
        display_portfolio_summary(account_dropdown.value, change['new'])
        display_securities(account_dropdown.value, change['new'])
        create_charts(account_dropdown.value, change['new'])

# Attach event handlers
account_dropdown.observe(on_account_change, names='value')
statement_dropdown.observe(on_statement_change, names='value')

# Display the interactive dashboard
dashboard = VBox([
    HTML('<h2 style="color: #007bff; text-align: center;">Financial Analytics Dashboard</h2>'),
    HTML('<p style="text-align: center; color: #666;">Select an account to view portfolio data. Accounts with data will show statements and securities.</p>'),
    HBox([account_dropdown, statement_dropdown]),
    summary_output,
    securities_output,
    charts_output
])

display(dashboard)

## Summary

This notebook provides:

1. **Interactive Dropdowns**: Select different banks/accounts and statement periods
2. **Portfolio Summary**: Key metrics for each statement period
3. **Securities Analysis**: Detailed breakdown of holdings with weights and returns
4. **Visual Charts**: Multiple chart types for portfolio analysis
5. **Trend Analysis**: Historical performance tracking over time

**Note**: Only accounts with uploaded portfolio statements will show data. If an account appears empty, you need to upload PDF statements for that account first.

Use the dropdowns above to explore different accounts and statement periods. The dashboard will automatically update to show relevant data and charts.